In [1]:
## read features:
import numpy as np
import pandas as pd

data_dir="/home/xin/Dropbox/BrainImaging_ML/data"
atlas_dir="/home/xin/Dropbox/BrainImaging_ML/Atlas_label"
work_dir="/home/xin/Dropbox/BrainImaging_ML/multidimensional_age_prediction/"


# ransac/huber with spm8 vbm features:
figure_dir = "/home/xin/Dropbox/BrainImaging_ML/multidimensional_age_prediction/figures_robustreg_behav_hc2"
result_dir = "/home/xin/Dropbox/BrainImaging_ML/multidimensional_age_prediction/result_robustreg_behav_hc2"

import os
from os import path
#trying to make shift_graphs directory if it does not already exist:

if not os.path.exists(figure_dir):
    os.mkdir(figure_dir)
    
if not os.path.exists(result_dir):
    os.mkdir(result_dir)


# vbm analysis of cat12, 856 subjects. 
#It contains subjects in the ptsd group. So we need to remove them in training stage.
# cat_vbm = pd.read_csv(data_dir+"/ROI_catROI_neuromorphometrics_Vgm.csv", delimiter = ",")
# drop features with all zero values:
# cat_vbm.drop(columns = ['lInfLatVen', 'lOC'], inplace = True)
# print("cat_vbm:")
# print(cat_vbm.shape)
# # print(cat_vbm[:5])
# vbm = cat_vbm

spm_vbm_aal = pd.read_csv(data_dir+"/GMV_AAL_AvgExtract.csv", delimiter = ",")
# rename the subject id:
spm_vbm_aal.iloc[:,0].replace(to_replace=\
                              r'/home/xin/BrainImaging2016/Gender/Result03_Nifti_T1/VBM8/resampled_img/rm0wp1',
                              value='', regex=True, inplace = True)

spm_vbm_aal.iloc[:,0].replace(to_replace=\
                              r'.nii.gz',
                              value='', regex=True, inplace = True)

spm_vbm_aal.rename(columns = {'Unnamed: 0':'names'}, inplace = True) 
spm_vbm_aal['names']=spm_vbm_aal['names'].astype(int)
# spm_vbm_aal.iloc[:,1:] = spm_vbm_aal.iloc[:,1:].apply(pd.to_numeric, errors='coerce')

# read roi labels for aal atlas:
aal_label = pd.read_csv(atlas_dir+"/AAL_116_Label.csv", delimiter = ',')
spm_vbm_aal.columns = ['names'] + aal_label['ROIname'].to_list()
vbm = spm_vbm_aal


## read label FA features:
label_fa = pd.read_csv(data_dir+"/WMlabelResults_FA_all.csv", index_col=[0])
label_fa = label_fa.drop(['Unnamed: 0.1'], axis = 1)

# remove the last column which is empty.
num_col_fa=len(label_fa.columns) 
label_fa=label_fa.iloc[:,:num_col_fa-1]

print("label fa:")
print(label_fa.shape)
# print(label_fa[:5])


## read tract FA features:
tract_fa = pd.read_csv(data_dir+"/WMtractResults_FA_all.csv", index_col=0)
tract_fa = tract_fa.drop(['Unnamed: 0.1'], axis = 1)
# remove the last column which is empty.
num_col_fa=len(tract_fa.columns) 
tract_fa=tract_fa.iloc[:,:num_col_fa-1]

print("tract fa:")
print(tract_fa.shape)
# print(tract_fa[:5])


# multimodal_features = pd.concat([cat_vbm, label_fa, tract_fa, label_md, tract_md, alff, reho], axis = 1,)

# multimodal_features_keys = pd.concat([cat_vbm.drop(['names'], axis=1), label_fa, tract_fa, label_md, tract_md, alff, reho], axis = 1,
#                              keys = ['gmv', 'label_fa', 'tract_fa', 'label_md', 'tract_md', 'alff', 'reho'])

multimodal_features = pd.concat([vbm, label_fa, tract_fa], axis = 1,)
subject_id = vbm['names'].to_frame()



label fa:
(856, 50)
tract fa:
(856, 20)


In [2]:
# label_fa.loc[:,'Cingulum.(cingulate.gyrus).L']
# print(label_fa.iloc[:,35])
# print(multimodal_features.iloc[:, 116+35+1 ])

# print(tract_fa.iloc[:,4])
# print(multimodal_features.iloc[:, 116+50+4+1])

In [3]:
cog_label_acc = [
    # executive control:
    "pcet_acc2",
    "pcpt_t_tp",
    "lnb_tp2",
    # episodic memory task:
    "pwmt_kiwrd_tot",
    "pfmt_ifac_tot",
    "volt_svt",
    # complex cognitive task:
    "pvrt_cr",
    "pmat_cr",
    "plot_tc",
    # social cognition task:
    "peit_cr",
    "pedt_a",
    "padt_a"
]

cog_label_speed = [
    # executive control:
    "pcet_rtcr",
    "pcpt_t_tprt",
    "lnb_rtc2",
    # episodic memory task:
    "pwmt_kiwrd_rtc",
    "pfmt_ifac_rtc",
    "volt_svtcrt",
    # complex cognitive task:
    "pvrt_rtcr",
    "pmat_rtcr",
    "plot_tcrt",
    # social cognition task:
    "peit_crt",
    "pedt_t",
    "padt_t",
    # sensorimotor speed task:
    "mp_mp2rtcr",
    "tap_tot"
]

cog_label_acc2 = [
    # executive control:
    "ABF",
    "ATT",
    "WM",
    # episodic memory task:
    "VMEM",
    "FMEM",
    "SMEM",
    # complex cognitive task:
    "LAN",
    "NVR",
    "SPA",
    # social cognition task:
    "EID",
    "EDI",
    "ADI"
]
cog_label_acc2 = [s + '_Accuracy' for s in cog_label_acc2]

cog_label_speed2 = [
    # executive control:
    "ABF",
    "ATT",
    "WM",
    # episodic memory task:
    "VMEM",
    "FMEM",
    "SMEM",
    # complex cognitive task:
    "LAN",
    "NVR",
    "SPA",
    # social cognition task:
    "EID",
    "EDI",
    "ADI",
    # sensorimotor speed task:
    "MOT",
    "SM"
]
cog_label_speed2 = [s + '_Speed' for s in cog_label_speed2]

cog_score_label = cog_label_acc + cog_label_speed
cog_score_label2 = cog_label_acc2 + cog_label_speed2
num_cog = len(cog_score_label)


In [4]:
import matplotlib.pyplot as plt

plt.plot(label_fa.iloc[:,35], tract_fa.iloc[:,4], 'b*')

In [5]:
# read subject_info:
# pnc_subject_info = pd.read_csv(data_dir+"/pnc_subject_info.csv", delimiter = ",")

# pnc_subject_info = pnc_subject_info[['SUBJID', 'Sex', 'age_at_cnb']]


# select mental healthy subjects:
behav_data = pd.read_csv(data_dir+"/pyresult01_pnc_behav_mental3.csv", delimiter = ",")
# hc_subject = hc_subject.loc[hc_subject['mental_hc3']==True, 'SUBJID']
behav_data_hc = behav_data.loc[behav_data['mental_hc2']==True, ['SUBJID', 'Sex', 'age_at_cnb']+cog_score_label]
behav_data_hc.columns = ['SUBJID', 'Sex', 'age_at_cnb']+cog_score_label2

behav_feature = behav_data_hc.merge(multimodal_features, how = "inner", 
                                            right_on = "names", left_on = "SUBJID")

# reverse speed scores:
behav_feature.loc[:,cog_label_speed2] = behav_feature.loc[:,cog_label_speed2]*-1

#subject_info_feature['Sex'] = subject_info_feature['Sex'].apply({'F':0, 'M':1}.get)
# code females as -1 for easy interpretation of beta for age and age2
behav_feature['Sex'] = behav_feature['Sex'].apply({'F':-1, 'M':1}.get)

# check merge
print(behav_feature.loc[:,['SUBJID', 'names']])
behav_feature.drop(['names'], axis=1, inplace = True)
# subject_info_feature.drop(['SUBJID'], axis=1, inplace = True)

#normalize the brain imaging feature columns:
behav_feature.iloc[:,3:] = (behav_feature.iloc[:,3:] - behav_feature.iloc[:,3:].min()) / \
                        (behav_feature.iloc[:,3:].max() - behav_feature.iloc[:,3:].min())

num_subjects = behav_feature.shape[0]


# count number of NaNs:
# row_nancount = subject_info_feature.isna().sum()
# subject_info_feature = subject_info_feature.loc[:, subject_info_feature.isna().sum()==0]



           SUBJID         names
0    6.000100e+11  600009963128
1    6.000209e+11  600020927179
2    6.000390e+11  600039015048
3    6.000526e+11  600052596653
4    6.001109e+11  600110937794
..            ...           ...
223  6.097148e+11  609714765360
224  6.097281e+11  609728145608
225  6.098028e+11  609802779962
226  6.098253e+11  609825254934
227  6.099877e+11  609987705265

[228 rows x 2 columns]


In [6]:
behav_feature.to_csv(result_dir+'/out01_behav_feature.csv')
behav_feature

,SUBJID,Sex,age_at_cnb,ABF_Accuracy,ATT_Accuracy,WM_Accuracy,VMEM_Accuracy,FMEM_Accuracy,SMEM_Accuracy,LAN_Accuracy,...,Inferior.fronto-occipital.fasciculus.L,Inferior.fronto-occipital.fasciculus.R,Inferior.longitudinal.fasciculus.L,Inferior.longitudinal.fasciculus.R,Superior.longitudinal.fasciculus.L,Superior.longitudinal.fasciculus.R,Uncinate.fasciculus.L,Uncinate.fasciculus.R,Superior.longitudinal.fasciculus.(temporal.part).L,Superior.longitudinal.fasciculus.(temporal.part).R
0,6.000100e+11,-1,9.0,0.820219,0.666667,0.6,0.947368,0.75,0.363636,0.916667,...,0.682056,0.768709,0.765607,0.710414,0.822855,0.672312,0.674132,0.664874,0.789640,0.904289
1,6.000209e+11,-1,15.0,0.249066,0.769231,0.8,0.842105,0.35,0.909091,0.833333,...,0.549499,0.745229,0.640513,0.725692,0.632849,0.692244,0.444163,0.470042,0.416445,0.678121
2,6.000390e+11,1,11.0,0.925080,0.846154,0.8,1.000000,0.55,0.818182,0.666667,...,0.335360,0.528781,0.397871,0.594044,0.249789,0.296193,0.347792,0.384577,0.513683,0.294175
3,6.000526e+11,-1,15.0,0.290341,0.923077,0.7,0.736842,0.65,0.818182,0.916667,...,0.516917,0.539925,0.518827,0.631980,0.540110,0.460032,0.152839,0.436512,0.752462,0.352849
4,6.001109e+11,-1,10.0,0.724404,0.615385,0.5,0.526316,0.00,0.272727,0.333333,...,0.573250,0.583742,0.521522,0.692858,0.330205,0.305825,0.441396,0.455438,0.545168,0.374684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,6.097148e+11,1,13.0,0.677270,0.897436,0.8,0.894737,0.80,0.545455,0.666667,...,0.737788,0.794829,0.694251,0.701686,0.451990,0.698653,0.452723,0.638607,0.363182,0.737912
224,6.097281e+11,1,18.0,0.679684,0.205128,0.8,1.000000,0.90,0.818182,0.750000,...,0.551190,0.641390,0.580176,0.653674,0.660195,0.811672,0.336769,0.462330,0.379010,0.602286
225,6.098028e+11,-1,16.0,0.804894,0.897436,0.8,0.842105,0.55,0.636364,0.666667,...,0.693127,0.820331,0.714007,0.758479,0.716400,0.577573,0.559471,0.763690,0.534994,0.216554
226,6.098253e+11,1,10.0,0.735641,0.666667,0.8,0.789474,1.00,0.909091,0.666667,...,0.332386,0.503468,0.402174,0.598515,0.293534,0.481289,0.447447,0.657864,0.330091,0.267813


In [7]:
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

def qudratic_r_squared_gender(Matrix):
    """
    compute the r squared of curve fit of chro_age vs. brain age.
    1. Use qudratic function to fit brain age with chronological age and gender.
    2. Then, compute r squared of brain age and predicted brain age with qudratic fit model.
    
    use the first 2 columns to fit the 3rd column of Matrix
    """
    
    # remove NaN values:
    nan_idx = np.isnan(Matrix).any(axis = 1)
    Matrix = Matrix[~nan_idx,:]
    
    chro_age = Matrix[:,1]
    gender = Matrix[:,0]
    brain_age = Matrix[:,2]
    
    print(Matrix.shape)
    if Matrix.shape[0]<20:
        output = np.empty((6))
        output[:] = np.nan
        return(output)
    else:
    
        def qudratic_fun(x, a, b, c, d, e, f):
            x1, x2 = x
            #return a + b*x1 + c*(x1**2) + d*x2 + e*x1*x2 f*(x1**2)*x2
            return a + b*x1 + c*(x1**2) + d*x2 + e*x1*x2 + f*(x1**2)*x2

        popt, pcov = curve_fit(qudratic_fun, (chro_age, gender), brain_age)
        brain_age_pred = qudratic_fun((chro_age, gender), popt[0], popt[1], popt[2], popt[3], popt[4], popt[5])

        r_square = r2_score(brain_age, brain_age_pred)

        return(np.array([popt[1], popt[2], popt[3], popt[4], popt[5], r_square]))
    

# def sigmoid(x, c, k, x0, y0):
#     return c / (1 + np.exp(-k*(x-x0))) + y0    

def sigmoid(x, k, x0):
    return 1 / (1 + np.exp(-k*(x-x0)))  

def sigmoid2(x, c, k, x0, y0):
    return c*x/((x-x0)**2+k)+y0

def sigmoid_fit(age, gender, feature):
    """
    compute the sigmoid curve fit of chro_age vs. brain feature.
    1. Use sigmoid function to fit feature with chronological age.
    2. Then, compute r squared of brain age and predicted brain age.
    
    use the first 2 columns to fit the 3rd column of Matrix
    """
    
    Matrix = np.transpose(np.vstack((age, feature, gender)))
    #print(Matrix)
    # remove NaN values:
    nan_idx = np.isnan(Matrix).any(axis = 1)
    Matrix = Matrix[~nan_idx,:]
    
    idx_male = Matrix[:,2]==1
    idx_female = Matrix[:,2]!=1
    #print(Matrix[idx_male,:])
    
    print(Matrix.shape)
    if Matrix.shape[0]<20:
        print('length of input matrix is less than 20, return nan values...')
        output = np.empty((6))
        output[:] = np.nan
        return(output)
    else:
    
#         popt, pcov = curve_fit(sigmoid, chro_age, feature, 
#                                maxfev=5000, method='dogbox', bounds=([0, -1, -1, 0],[1, 1, 1, 1]))
        
        popt, pcov = curve_fit(sigmoid2, Matrix[idx_male,0], Matrix[idx_male,1], \
                               maxfev=5000, method='dogbox', bounds=([-50, 10, 10, -1],[50, 300, 100, 1.5]))
        
        popt2, pcov2 = curve_fit(sigmoid2, Matrix[idx_female,0], Matrix[idx_female,1], \
                               maxfev=5000, method='dogbox', bounds=([-50, 10, 10, -1],[50, 300, 100, 1.5]))
        
        feature_pred = Matrix[:,0]*np.nan
        feature_pred[idx_male] = sigmoid2(Matrix[idx_male,0], popt[0], popt[1], popt[2], popt[3])
        feature_pred[idx_female] = sigmoid2(Matrix[idx_female,0], popt2[0], popt2[1], popt2[2], popt2[3])

        r_square = r2_score(chro_age, feature_pred)

        return(np.array([popt[0], popt[1], popt[2], popt[3],\
                         popt2[0], popt2[1], popt2[2], popt2[3],\
                         r_square]))
        
   


In [8]:
# test sigmoid curve:

# x = np.linspace(0, 30, num=30)
# y_fitted = sigmoid(x, feature_beta_df.iloc[i_feature,1],feature_beta_df.iloc[i_feature,2],
#                    feature_beta_df.iloc[i_feature,3],feature_beta_df.iloc[i_feature,4])


In [9]:
# from sklearn.covariance import MinCovDet
# from sklearn.ensemble import IsolationForest

# def find_outlier(X, thresh_std = 2.5):
    
#     # fit a Minimum Covariance Determinant (MCD) robust estimator to data
#     robust_cov = MinCovDet().fit(X)
#     robust_mahal = robust_cov.mahalanobis(X - robust_cov.location_) ** (0.33)
    
#     m_mahal = np.mean(robust_mahal)
#     sd_mahal = np.std(robust_mahal)
#     thresh_high = m_mahal+sd_mahal
#     thresh_low = m_mahal-sd_mahal
    
#     remove_idx = np.logical_or(robust_mahal>thresh_high, robust_mahal<thresh_low)
    
#     return(remove_idx)

# rng = np.random.RandomState(42)

# def find_outlier(X):
#     clf = IsolationForest(behaviour = 'new', max_samples=100, random_state=rng, contamination=.01)
#     clf.fit(X)
#     y_pred_outliers = clf.predict(X)
#     y_pred_outliers = y_pred_outliers==-1
    
#     return(y_pred_outliers)

def find_outlier(x, thresh_std = 10):
    mean = np.mean(x)
    std = np.std(x)
    
    outlier = np.logical_or(x>(mean+thresh_std*std), x<(mean-thresh_std*std))
    return(outlier)
    

In [10]:
features_keys = pd.concat([behav_data_hc.iloc[:, 3:], vbm.drop(['names'], axis=1), label_fa, tract_fa], axis = 1,
                             keys = ['behav', 'gmv', 'label_fa', 'tract_fa'])

features_keys = features_keys.columns.get_level_values(0)
features_keys


feature_beta_df = pd.DataFrame(index = behav_feature.columns[3:], 
                               columns = ['age', 'age$^2$', 'gender', 'age*gender', 'age*gender$^2$', 'Rsquare'])

# feature_beta_df = pd.DataFrame(index = multimodal_features.columns[1:], 
#                                columns = ['c_m', 'k_m', 'x0_m', 'y0_m',\
#                                           'c_f', 'k_f', 'x0_f', 'y0_f',\
#                                           'Rsquare'])

feature_beta_df.insert(0, 'modality', features_keys)

feature_p_df = pd.DataFrame(index = behav_feature.columns[3:], 
                               columns = ['age', 'age$^2$', 'gender', 'age*gender', 'age*gender$^2$'])
feature_p_df.insert(0, 'modality', features_keys)


Rsquare_col = feature_beta_df.shape[1]


In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import LinearRegression
from scipy import stats


# scaler = MinMaxScaler()
scaler = RobustScaler()

# scaler.fit(subject_info_feature.iloc[:,3:].values)
# subject_info_feature.iloc[:,3:] = scaler.transform(subject_info_feature.iloc[:,3:].values)

# alpha = np.power(10, np.linspace(-1, 3, num = 50))
# ridge_cv = RidgeCV(alphas=alpha, cv = 5)

lr = LinearRegression()

# as robust scaler scales the data by IQR, we set the residualthreshold to 1.5
robust_reg = RANSACRegressor(base_estimator = lr, min_samples=.8, residual_threshold = 1.5,
                             stop_probability=.99, stop_n_inliers = num_subjects*.9)
# robust_reg = HuberRegressor(epsilon=1.35, max_iter=100, alpha=0.0001, warm_start=False, fit_intercept=True, tol=1e-05)

num_features = len(behav_feature.columns)-3

scaler.fit(behav_feature.iloc[:,3:].values)
behav_feature.iloc[:,3:] = scaler.transform(behav_feature.iloc[:,3:].values)

for i_feature in range(num_features):
    print(i_feature)
    print(feature_beta_df.index[i_feature])
    
    gender = behav_feature.iloc[:,1].values
    chro_age = behav_feature.iloc[:,2].values
    
    feature = behav_feature.iloc[:,3+i_feature].values
    nan_idx = np.isnan(feature)
    
#     # remove outliers:
#     X = np.transpose(np.vstack((chro_age, feature, gender)))
#     remove_idx = find_outlier(X)
    gender = gender[~nan_idx]
    chro_age = chro_age[~nan_idx]
    
    X = np.transpose(np.vstack((chro_age, chro_age**2, gender, chro_age*gender, (chro_age**2)*gender)))
    y = feature[~nan_idx]
    
############################################## ridge CV:########################################

#     # remove outliers based on the residuals of ridge regression:
#     clf = ridge_cv.fit(X, y)
#     residuals = y-clf.predict(X)
#     remove_idx = find_outlier(residuals)
    
#     # scale y with outlier removed:
#     scaler.fit(y[~remove_idx].reshape(-1,1))
#     y = scaler.transform(y.reshape(-1,1))
    
#     clf = ridge_cv.fit(X[~remove_idx,:], y[~remove_idx].flatten())
#     r2 = clf.score(X[~remove_idx,:], y[~remove_idx]) 
#     feature_beta_df.iloc[i_feature,1:] = np.hstack((clf.coef_, r2))

############################################ robust regression: #################################

    # scale the feature with inliers:
#     robust_reg.fit(X, y)
#     inlier_mask = np.logical_not(robust_reg.outliers_)
#     inlier_mask = robust_reg.inlier_mask_
#     scaler.fit(y[inlier_mask].reshape(-1,1))
#     y = scaler.transform(y.reshape(-1,1)).flatten()

    lr_all = lr.fit(X, y)
    # robust regression:
    robust_reg.fit(X, y)
    
#     inlier_mask = np.logical_not(robust_reg.outliers_)
    inlier_mask = robust_reg.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)
    coefs = robust_reg.estimator_.coef_
    
    # fit linear regression inliers and get the p-values of coefficients:
    lr.fit(X[inlier_mask,:], y[inlier_mask])
    params = np.append(lr.intercept_,lr.coef_)
    predictions = lr.predict(X[inlier_mask,:])
    
    # calculate p values:
    newX = np.append(np.ones((len(X[inlier_mask,:]),1)), X[inlier_mask,:], axis=1)
    MSE = (sum((y[inlier_mask]-predictions)**2))/(len(newX)-len(newX[0]))

    var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b

    p_values =np.array([2*(1-stats.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b])
    
    # compare the coefs of lr (with inliers) and ransac, which should be same
    print(coefs)
    print(lr.coef_)
    
    # set non-significant coefs to 0:
    # coefs[p_values[1:]>0.05]==0 # it's weird that this does not work!!!!
    # np.place(coefs, p_values[1:]>.05, 0)
    
    r2 = robust_reg.score(X, y) 
    feature_p_df.iloc[i_feature,1:] = p_values[1:]
    feature_beta_df.iloc[i_feature,1:] = np.hstack((coefs, r2))
#     feature_beta_df.iloc[i_feature,1:] = np.hstack((robust_reg.coef_, r2))
    
############################################# make regression plots:#############################
    plot_age = np.arange(5,26, step = 1)
    plot_gender = np.ones(21)
    
    plot_x_male = np.transpose(np.vstack((plot_age, plot_age**2, plot_gender, 
                                          plot_age*plot_gender, (plot_age**2)*plot_gender)))
    
    plot_x_female = np.transpose(np.vstack((plot_age, plot_age**2, -plot_gender,
                                            plot_age*-plot_gender, (plot_age**2)*-plot_gender)))
    
    y_curve_male = lr_all.predict(plot_x_male)
    y_curve_female = lr_all.predict(plot_x_female)
    
    y_curve_male = lr_all.predict(plot_x_male)
    y_curve_female = lr_all.predict(plot_x_female)
    
    y_curve_male_ransac = robust_reg.predict(plot_x_male)
    y_curve_female_ransac = robust_reg.predict(plot_x_female)
    
    y_curve_male_ransac = robust_reg.predict(plot_x_male)
    y_curve_female_ransac = robust_reg.predict(plot_x_female)
    
    label = "$R^2$ = %.3f" % r2

    fig = plt.figure()
    ax = fig.add_subplot(111)    
    
    ax.plot(plot_age, y_curve_female, 'r--')
    ax.plot(plot_age, y_curve_male, 'b--')
    
    ax.plot(plot_age, y_curve_female_ransac, 'r-')
    ax.plot(plot_age, y_curve_male_ransac, 'b-')
    
    idx = np.logical_and(gender==-1, outlier_mask)
    ax.plot(chro_age[idx]+np.random.random(sum(idx))*.3, y[idx], 
            'r+', markersize = 2.5)
    
    idx = np.logical_and(gender==-1, inlier_mask)
    ax.plot(chro_age[idx]+np.random.random(sum(idx))*.3, y[idx], 
            'ro', label='females', markersize = 2.5)
    
    idx = np.logical_and(gender==1, outlier_mask)
    ax.plot(chro_age[idx]+np.random.random(sum(idx))*.3, y[idx], 
            'b+', markersize = 2.5)
    
    idx = np.logical_and(gender==1, inlier_mask)
    ax.plot(chro_age[idx]+np.random.random(sum(idx))*.3, y[idx], 
            'bo', label='males', markersize = 2.5)
    
    ax.text(8, max(y), label, fontsize = 10)
    y_label = feature_beta_df.index[i_feature]+' ('+ feature_beta_df.iloc[i_feature, 0]+')'    
    if len(y_label)>20:
        y_label = y_label[:20]

    ax.set_ylabel(y_label)

    ax.legend()
    fig.savefig(figure_dir+'/figure_scatter_age_feature_' +
               feature_beta_df.iloc[i_feature, 0] + str(i_feature) + '.jpg', format = 'jpg', dpi=300)
    plt.close('all')

        
############################################## sigmoid fit:########################################
#     try:
#         feature_beta_df.iloc[i_feature,1:] = sigmoid_fit(chro_age, gender, feature)

#         x = np.linspace(6, 23, num=30)
#         y_fit_m = sigmoid2(x, feature_beta_df.iloc[i_feature,1],feature_beta_df.iloc[i_feature,2],
#                            feature_beta_df.iloc[i_feature,3],feature_beta_df.iloc[i_feature,4])

#         y_fit_f = sigmoid2(x, feature_beta_df.iloc[i_feature,5],feature_beta_df.iloc[i_feature,6],
#                            feature_beta_df.iloc[i_feature,7],feature_beta_df.iloc[i_feature,8])

#         fig = plt.figure()
#         ax = fig.add_subplot(111)    
#         ax.plot(x, y_fit_m, 'b--', label='male')
#         ax.plot(x, y_fit_f, 'r--', label='female')

#         male_idx = subject_info_feature.iloc[:,1]==1
#         female_idx = subject_info_feature.iloc[:,1]!=1

#         ax.plot(subject_info_feature.loc[male_idx].iloc[:,2],
#                 subject_info_feature.loc[male_idx].iloc[:,i_feature+3], 'bo')

#         ax.plot(subject_info_feature.loc[female_idx].iloc[:,2],
#                 subject_info_feature.loc[female_idx].iloc[:,i_feature+3],'ro')

#         ax.legend()
#         fig.savefig(figure_dir+'/figure_scatter_age_feature_sigmoid2fit_' +
#                    feature_beta_df.iloc[i_feature, 0] + str(i_feature) + '.jpg', format = 'jpg', dpi=300)
#         plt.close('all')

#         print(feature_beta_df.iloc[i_feature,1:])
#     except:
#         fig = plt.figure()
#         ax = fig.add_subplot(111)    

#         male_idx = subject_info_feature.iloc[:,1]==1
#         female_idx = subject_info_feature.iloc[:,1]!=1

#         ax.plot(subject_info_feature.loc[male_idx].iloc[:,2],
#                 subject_info_feature.loc[male_idx].iloc[:,i_feature+3], 'bo')

#         ax.plot(subject_info_feature.loc[female_idx].iloc[:,2],
#                 subject_info_feature.loc[female_idx].iloc[:,i_feature+3],'ro')

#         ax.legend()
#         fig.savefig(figure_dir+'/figure_scatter_age_feature_sigmoid2fit_fail_' +
#                    feature_beta_df.iloc[i_feature, 0] + str(i_feature) + '.jpg', format = 'jpg', dpi=300)
#         plt.close('all')
    
    

0
ABF_Accuracy
[ 0.1819079  -0.00502352 -1.7357813   0.28088261 -0.00989086]
[ 0.1819079  -0.00502352 -1.7357813   0.28088261 -0.00989086]
1
ATT_Accuracy
[ 0.3088299  -0.00771059  0.9243461  -0.08780311  0.00204701]
[ 0.3088299  -0.00771059  0.9243461  -0.08780311  0.00204701]
2
WM_Accuracy
[ 0.32551747 -0.00890664 -1.35652825  0.20405585 -0.00706175]
[ 0.32551747 -0.00890664 -1.35652825  0.20405585 -0.00706175]
3
VMEM_Accuracy
[ 0.1553243  -0.00478078 -0.59376127  0.08345861 -0.00309192]
[ 0.1553243  -0.00478078 -0.59376127  0.08345861 -0.00309192]
4
FMEM_Accuracy
[ 0.40217559 -0.01147521 -1.42660419  0.22449852 -0.00826275]
[ 0.40217559 -0.01147521 -1.42660419  0.22449852 -0.00826275]
5
SMEM_Accuracy
[ 0.10219984 -0.00226633  0.63633875 -0.07578852  0.00245831]
[ 0.10219984 -0.00226633  0.63633875 -0.07578852  0.00245831]
6
LAN_Accuracy
[ 0.45826435 -0.01268784 -0.64852499  0.12468006 -0.00460873]
[ 0.45826435 -0.01268784 -0.64852499  0.12468006 -0.00460873]
7
NVR_Accuracy
[ 0.364173

56
Left Anterior Cingulum
[-1.03871821e-01  1.10963957e-03  1.36444839e+00 -1.76704215e-01
  5.81353113e-03]
[-1.03871821e-01  1.10963957e-03  1.36444839e+00 -1.76704215e-01
  5.81353113e-03]
57
Right Anterior Cingulum
[-9.00306135e-02 -3.44082791e-05  1.39642208e+00 -1.89049023e-01
  6.48555997e-03]
[-9.00306135e-02 -3.44082791e-05  1.39642208e+00 -1.89049023e-01
  6.48555997e-03]
58
Left Middle Cingulum
[-0.37260545  0.00938045  0.28411891 -0.02558697  0.00116697]
[-0.37260545  0.00938045  0.28411891 -0.02558697  0.00116697]
59
Right Middle Cingulum
[-0.3053449   0.00648182  0.59773508 -0.05122813  0.00163038]
[-0.3053449   0.00648182  0.59773508 -0.05122813  0.00163038]
60
Left Cingulum
[-2.15847995e-01  4.85592296e-03  7.31315793e-01 -3.74176065e-02
  6.74457680e-04]
[-2.15847995e-01  4.85592296e-03  7.31315793e-01 -3.74176065e-02
  6.74457680e-04]
61
Right Cingulum
[-0.2767655   0.00708295  0.96567623 -0.09054034  0.00255214]
[-0.2767655   0.00708295  0.96567623 -0.09054034  0.002

111
Right Middle Temporal
[-0.36353382  0.00888959  0.8110229  -0.08564461  0.00256928]
[-0.36353382  0.00888959  0.8110229  -0.08564461  0.00256928]
112
Left Middle Temporal Pole
[-0.1415551   0.00372622  0.35163602 -0.03752382  0.00130752]
[-0.1415551   0.00372622  0.35163602 -0.03752382  0.00130752]
113
Right Middle Temporal Pole
[-0.30899391  0.00911505  2.14542351 -0.3216714   0.01178073]
[-0.30899391  0.00911505  2.14542351 -0.3216714   0.01178073]
114
Left Inferior Temporal
[-0.17379779  0.00333425  0.82726421 -0.10995287  0.00386243]
[-0.17379779  0.00333425  0.82726421 -0.10995287  0.00386243]
115
Right Inferior Temporal
[-0.28392219  0.00633454  0.67269889 -0.08143149  0.00279621]
[-0.28392219  0.00633454  0.67269889 -0.08143149  0.00279621]
116
Cerebellum_Crus1_L
[-6.97759314e-02  4.19314226e-04  6.35318328e-01 -6.95416071e-02
  1.13937887e-03]
[-6.97759314e-02  4.19314226e-04  6.35318328e-01 -6.95416071e-02
  1.13937887e-03]
117
Cerebellum_Crus1_R
[-1.40819963e-01  2.426779

164
 Anterior.corona.radiata.R
[ 0.37059548 -0.01094117  1.31549093 -0.19895883  0.00733929]
[ 0.37059548 -0.01094117  1.31549093 -0.19895883  0.00733929]
165
 Anterior.corona.radiata.L
[ 0.42191266 -0.01257836  0.73175116 -0.08954858  0.00310986]
[ 0.42191266 -0.01257836  0.73175116 -0.08954858  0.00310986]
166
 Superior.corona.radiata.R
[ 0.28460727 -0.007422   -0.39687828  0.04363375 -0.00074778]
[ 0.28460727 -0.007422   -0.39687828  0.04363375 -0.00074778]
167
 Superior.corona.radiata.L
[ 0.29455337 -0.00793853  0.31967428 -0.05579966  0.00233378]
[ 0.29455337 -0.00793853  0.31967428 -0.05579966  0.00233378]
168
 Posterior.corona.radiata.R
[ 0.34850679 -0.0101783  -0.17013092  0.01278539  0.00043832]
[ 0.34850679 -0.0101783  -0.17013092  0.01278539  0.00043832]
169
 Posterior.corona.radiata.L
[ 0.28115441 -0.00825247 -0.26789493  0.01590876  0.00028413]
[ 0.28115441 -0.00825247 -0.26789493  0.01590876  0.00028413]
170
 Posterior.thalamic.radiation.(include.optic.radiation).R
[ 0.25

In [12]:
# feature_beta_df.to_csv(result_dir+'/feature_beta_sigmoid2.csv')
feature_beta_df.to_csv(result_dir+'/behav_feature_beta_ransac_lr.csv')
feature_p_df.to_csv(result_dir+'/behav_feature_p_ransac_lr.csv')
# feature_beta_df.to_csv(result_dir+'/feature_beta_huber.csv')

In [13]:
# clf.alpha_

In [14]:
# x=np.array([[1,2,3],[np.nan, 2,4]])
# np.isnan(x).any(axis=1)
feature_beta_df.iloc[22,1:].isnull().any()

False

In [15]:
feature_beta_df

,modality,age,age$^2$,gender,age*gender,age*gender$^2$,Rsquare
ABF_Accuracy,behav,0.181908,-0.00502352,-1.73578,0.280883,-0.00989086,0.0680203
ATT_Accuracy,behav,0.30883,-0.00771059,0.924346,-0.0878031,0.00204701,0.188576
WM_Accuracy,behav,0.325517,-0.00890664,-1.35653,0.204056,-0.00706175,0.146375
VMEM_Accuracy,behav,0.155324,-0.00478078,-0.593761,0.0834586,-0.00309192,0.0406479
FMEM_Accuracy,behav,0.402176,-0.0114752,-1.4266,0.224499,-0.00826275,0.128487
...,...,...,...,...,...,...,...
Superior.longitudinal.fasciculus.R,tract_fa,0.372971,-0.0105051,0.0735974,-0.0163551,0.00133869,0.191736
Uncinate.fasciculus.L,tract_fa,0.387803,-0.0118244,0.662346,-0.110195,0.00454675,0.115589
Uncinate.fasciculus.R,tract_fa,0.353951,-0.0103687,-0.0344704,-0.0207193,0.00171614,0.127418
Superior.longitudinal.fasciculus.(temporal.part).L,tract_fa,0.309371,-0.00877603,0.562019,-0.0869563,0.00323578,0.0305486
